In [28]:
#Lesson 9: Real-World Application : Performance of main asset classes
import yfinance as yf
import pandas as pd
import numpy as np
import math as ma

In [29]:
#Ask Whether The Risk Free Rate is Given or If It Must Be Calculated From FF3F Data

import pandas_datareader as pdr

while True:
    # Prompt the user and immediately handle the response
    rf_Prompt = input("Has the risk-free rate been provided already? (Yes/No): ").strip().lower()

    if rf_Prompt == "yes":
        rf_Rate = float(input("Input the given risk-free rate here: "))  # ensure input is converted to float for calculations
        print("Risk Free Rate is: ", rf_Rate)
        break
    elif rf_Prompt == "no":
        ff3f_Start = input("Enter the start date for pulling FF3F data here [Format like 1950-01-01]: ")
        ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', ff3f_Start, '2024-12-31')[0]/100
        rf_Rate = ff3f['RF'].mean()  # Assuming ff3f['RF'] is already loaded and contains the relevant data
        print("The FF3F monthly risk free rate is:  ", rf_Rate)
        break
    else:
        print("Not a valid input! Please say 'Yes' or 'No'.")


Has the risk-free rate been provided already? (Yes/No):  Yes
Input the given risk-free rate here:  0.0001


Risk Free Rate is:  0.0001


In [30]:

size = int(input("Enter how many stocks will be in your portfolio here: "))

portfolio = [None] * size

for i in range(size):
    portfolio[i] = input(f"Enter the ticker for stock number {i+1} here: ")
    
print("Portfolio: ", portfolio)

start = input("Start data [Format like '2010-01-01']: ") #Start Date For yfinance data pull
end = input("End date [Format like '2020-01-01'] (Best to always get an extra months data for more accurate results: ") #End data for yfinance data pull

m_data = yf.download(portfolio, start = start, end = end, interval = '1mo')['Adj Close'].dropna()



m_data = m_data[portfolio]

m_data = m_data.pct_change().dropna()



m_data.head(5)

#Estimated Returns, Standard Deviations, Risk Premium

average_Data = m_data.mean()

standard_Dev = m_data.std() #Gets standard deviation from return data

risk_Premia = average_Data - rf_Rate




Enter how many stocks will be in your portfolio here:  3
Enter the ticker for stock number 1 here:  TSLA
Enter the ticker for stock number 2 here:  AMZN
Enter the ticker for stock number 3 here:  MSFT


Portfolio:  ['TSLA', 'AMZN', 'MSFT']


Start data [Format like '2010-01-01']:  2016-01-01
End date [Format like '2020-01-01'] (Best to always get an extra months data for more accurate results:  2021-01-01


[*********************100%%**********************]  3 of 3 completed


In [31]:
#Sigma Covariance Matrix and its inverse

df = pd.DataFrame(m_data) #Takes stocks returns and converts it into a df datafield

cov_matrix = df.cov() #Calculates the covar matrix

inverse = np.linalg.inv(cov_matrix)

print(f"Covariance Matrix: \n", cov_matrix)

Covariance Matrix: 
 Ticker      TSLA      AMZN      MSFT
Ticker                              
TSLA    0.036837  0.005131  0.004074
AMZN    0.005131  0.006508  0.002795
MSFT    0.004074  0.002795  0.002805


In [32]:
#W_Tangeant Weights
numerator = np.dot(inverse, risk_Premia) #Numerator for w_tangent

denominator = np.dot(np.ones(len(risk_Premia)), numerator)

w_tangent = (numerator / denominator) * 100

for i in range(size):
    print(f"The W-Tan for {portfolio[i]} is {w_tangent[i].round(2)}%")

The W-Tan for TSLA is 9.04%
The W-Tan for AMZN is 14.33%
The W-Tan for MSFT is 76.63%


In [33]:
#Finding Risk Premium and Variance of Portfolio
w_tangent = w_tangent / 100

risk_Premia = np.array(risk_Premia)

#Portfolio Risk Premium
portfolio_RP = (risk_Premia @ w_tangent)


#Variance of Portfolio
portfolio_variance = np.dot(w_tangent.T, np.dot(cov_matrix.values, w_tangent))
print(f"The variance of the portfolio is: {portfolio_variance.round(7)}")

The variance of the portfolio is: 0.0033931


In [34]:
risk_Aversion = input("Input the risk-aversion coefficient here: ")
risk_Aversion = int(risk_Aversion)

w_OCA = portfolio_RP/ (portfolio_variance * risk_Aversion)

print(f"The weighted optimal capital allocation is: {w_OCA}")

Input the risk-aversion coefficient here:  5


The weighted optimal capital allocation is: 1.8352954558152885


In [35]:
#W_oca Tangent Weights
w_tan_weights = [None] * size
rf_Alloc = (1 - w_OCA) * 100

for i in range(size):
    w_tan_weights[i] = w_OCA * w_tangent[i]
    w_tan_weights[i] = w_tan_weights[i].round(3)
    print(f" w_OCA tangent weight for {portfolio[i]} is {w_tan_weights[i]*100}%")
    
    
print(f"\nThe risk free rate allocation: {rf_Alloc.round(3)}%") #Rf-Rate Allocated (How much will we borrow)

 w_OCA tangent weight for TSLA is 16.6%
 w_OCA tangent weight for AMZN is 26.3%
 w_OCA tangent weight for MSFT is 140.6%

The risk free rate allocation: -83.53%


In [36]:
#Sharpe Ratios for individual stocks and the portfolio

stock_Sharpes = [None] * size
for i in range(size):
    stock_Sharpes[i] = (average_Data[i] - rf_Rate) / standard_Dev[i]
    print(f" The Sharpe Ratio for {portfolio[i]}: {stock_Sharpes[i]}") #Indiv Sharpes




 The Sharpe Ratio for TSLA: 0.3435851286977536
 The Sharpe Ratio for AMZN: 0.40247739555926776
 The Sharpe Ratio for MSFT: 0.505650443678576


C:\Users\Sean\AppData\Local\Temp\ipykernel_9376\752195567.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stock_Sharpes[i] = (average_Data[i] - rf_Rate) / standard_Dev[i]


In [37]:
#Sharpe Ratio of Portfolio
port_Sharpe = portfolio_RP / portfolio_variance ** 0.5
print(f"The Sharpe Ratio of the portfolio: {port_Sharpe}")

The Sharpe Ratio of the portfolio: 0.5345288667681009
